# Etude Snorkel / Extration de partenariats d'entreprises
Pour ce projet, nous allons essayer d'extraire des relations de partenariats économiques entre des entreprises depuis des articles. <br/>
Le but étant de comprendre l'utilisation de snorkel.

Nous allons découper notre traitement dans 3 notebooks (chacun correspondant à une étape du pipeline `Snorkel` :
1. Preprocessing
2. Training
3. Evaluation

## Partie I: Corpus Preprocessing

Dans ce notebook, nous allons pré-traiter plusieurs documents en utilisant la solution `Snorkel`.
Nous allons commencer par "parser" les documents pour construire notre hiérarchie de _contexts_.
Nous allons ensuite instancier nos _candidats_ qui correspondent aux objets que nous voulons classer. Dans notre cas identifier les relations de partenariat entre 2 entreprises.
***

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import pandas as pd
import pickle

from snorkel.models import StableLabel
from snorkel.db_helpers import reload_annotator_labels
from snorkel.parser import TSVDocPreprocessor
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence, candidate_subclass
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import OrganizationMatcher, DictionaryMatch
from snorkel.viewer import SentenceNgramViewer

# Connect to the database backend and initalize a Snorkel session
from snorkel import SnorkelSession
session = SnorkelSession()

# I. Chargement des données

## 1. Chargement des noms d'entreprises

Les entreprises à étudier ont été stockées dans un fichier `pickle` dans l'étape **préparation des données** (`prepare_data.py`)
On commence par charger ces noms des entreprises utiles pour le parsing.

In [2]:
with open('./data/companies.pkl', 'rb') as f:
    company_names = pickle.load(f)
print("Nombre d'entreprises", len(company_names))

Nombre d'entreprises 868


## 2. Chargement des découpages des données

Dans l'étape **préparation des données** (`prepare_data.py`) nous avons aussi réparti nos données en 3 ensembles : entrainement, développement et tests. Chargons cette répartition stockée dans des fichiers `pickle`.

In [3]:
with open('./data/train_set.pkl', 'rb') as f:
    train_set = pickle.load(f)
with open('./data/dev_set.pkl', 'rb') as f:
    dev_set = pickle.load(f)
with open('./data/test_set.pkl', 'rb') as f:
    test_set = pickle.load(f)
print("train set length :", len(train_set), 'documents')
print("dev set length   :", len(dev_set), 'documents')
print("test set length  :", len(test_set), 'documents')

train set length : 368 documents
dev set length   : 45 documents
test set length  : 45 documents


# II. Parsing de nos articles

## 1. Configuration d'un `DocPreprocessor`

Nous allons charger le corpus de documents (dans notre cas les articles) et effectuer les pré-traitements sur les données. <br/>
Nos articles  sont stockés sous la forme : nom du document et texte associé.<br/>
Nous allons utiliser la classe `TSVDocPreprocessor` pour lire ces documents.

In [4]:
n_docs = 1000
doc_preprocessor = TSVDocPreprocessor('data/articles.tsv', max_docs=n_docs)

## 2. Creation d'un `CorpusParser`

Nous allons ensuite nous appuyer sur la librairie [Spacy](https://spacy.io/), un parser NLP, afin de découper nos documents en phrases et tokens et appliquer un NER (Named-entity Recognition) : classer les entités nommés dans des catégories (ex : Personne, Organisation, ...).

In [5]:
corpus_parser = CorpusParser(parser=Spacy())
%time corpus_parser.apply(doc_preprocessor, count=n_docs)

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 7.02 s, sys: 931 ms, total: 7.95 s
Wall time: 8.89 s


Les données sont stockées dans une base SQLLite par Snorkel. Nous pouvons faire quelques requêtes pour vérifier que nos données sont correctement enregistrées.

In [6]:
print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

Documents: 458
Sentences: 472


# III. Génération des candidats

Dans cette partie nous allons extraire les _candidats__ de notre corpus. Ce sont les objets pour lesquels nous souhaitons faire une prédiction. 
Dans notre cas, un candidat est une paire de nom d'entreprise trouvée dans une phrase et pour lesquels nous voulons prédire s'ils sont partenaires ou non.

## 1. Définition du schéma de candidat

Nous avons défini une relation binaire _competitor_ qui associe deux objets texte de type `Span`.

In [7]:
Partner = candidate_subclass('Partner', ['company1', 'company2'])

## 2. Extraction des candidats

Nous allons maintenant extraire nos candidats en identifiant dans chaque phrase, les paires de n-grams (jusqu'à 6) que l'on retrouve dans notre dictionnaire de noms d'entreprises.

In [8]:
ngrams         = Ngrams(n_max=6)
company_matcher = DictionaryMatch(d=company_names, ignore_case=True)
cand_extractor = CandidateExtractor(Partner, [ngrams, ngrams], [company_matcher, company_matcher])

Ensuite nous découpons nos données en jeu d'entrainement (80%), de developpements (10%) et de tests (10%) en nous basons sur le découpage pré-établie dans la phase préparation des données.

In [9]:
docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if (doc.name in dev_set) :
            dev_sents.add(s)
        elif (doc.name in test_set) :
            test_sents.add(s)      
        else:
            train_sents.add(s)

Nous appliquons ensuite notre extracteur de candidats sur les 3 jeux de données.

In [10]:
%%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    cand_extractor.apply(sents, split=i)
    print("Number of candidates:", session.query(Partner).filter(Partner.split == i).count())

Clearing existing...
Running UDF...
[========================================] 100%

Number of candidates: 2464
Clearing existing...
Running UDF...
[========================================] 100%

Number of candidates: 166
Clearing existing...
Running UDF...
[========================================] 100%

Number of candidates: 304
CPU times: user 8.02 s, sys: 329 ms, total: 8.35 s
Wall time: 8.81 s


# III. Chargement des données labelisées

- Nous enregistrons maintenant les données qui ont été identifiés manuellement comme des partenaires. <br/>
- Ces données sont appelés les gold-labels et ils nous serviront à évaluer nos algorithmes.<br/>
- Nous mettons ensuite à jour les annotations pour les données de développement et de test.

In [11]:
FPATH = 'data/gold_labels.tsv'

def load_external_labels(session, candidate_class, annotator_name='gold'):
    gold_labels = pd.read_csv(FPATH, sep="\t")
    for index, row in gold_labels.iterrows(): 
        # We check if the label already exists, in case this cell was already executed
        context_stable_ids = "~~".join([row['company1'], row['company2']])
    
        query = session.query(StableLabel).filter(StableLabel.context_stable_ids == context_stable_ids)
        query = query.filter(StableLabel.annotator_name == annotator_name)
        if query.count() == 0:
            session.add(StableLabel(
                context_stable_ids=context_stable_ids,
                annotator_name=annotator_name,
                value=row['label']))
                    
        # Because it's a symmetric relation, load both directions...
        context_stable_ids = "~~".join([row['company2'], row['company1']])
        
        query = session.query(StableLabel).filter(StableLabel.context_stable_ids == context_stable_ids)
        query = query.filter(StableLabel.annotator_name == annotator_name)
        if query.count() == 0:
            session.add(StableLabel(
                context_stable_ids=context_stable_ids,
                annotator_name=annotator_name,
                value=row['label']))
    # Commit session
    session.commit()
    
    # Reload annotator labels
    reload_annotator_labels(session, candidate_class, annotator_name, split=1, filter_label_split=False)
    reload_annotator_labels(session, candidate_class, annotator_name, split=2, filter_label_split=False)

In [12]:
%time missed = load_external_labels(session, Partner, annotator_name='gold')

AnnotatorLabels created: 103
AnnotatorLabels created: 181
CPU times: user 47.2 s, sys: 907 ms, total: 48.1 s
Wall time: 52.5 s


# IV. Contrôle du chargement des données

Nous faisons quelques requêtes en base pour s'assurer que nos données sont correctement enregistrées.

In [13]:
cands = session.query(Partner).filter(Partner.split == 1).all()

In [14]:
print (cands[0].company1.get_attrib_tokens(a='words'))
print (cands[0].company2.get_attrib_tokens(a='words'))

['Atos']
['VMware']


In [15]:
sentence = cands[0].get_parent()
print("- Sentence : ", sentence.text)
document = sentence.get_parent()
print("- Doc Id = ", document.name)

- Sentence :  Atos has developed a multivendor alliancewith Dell EMC, Intel, Juniper Networks, Red Hat and VMware, aiming to accelerate network functions virtualization (NFV) deployments in service provider networks

- Doc Id =  8eb02e8d-ff02-489f-8fa0-bb0036ef5008


### Tests des `Labeling Functions` helpers

In [16]:
import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,get_text_splits,
    contains_token, rule_text_btw, rule_text_in_span
)

In [17]:
my_candidate = cands[0]

In [18]:
get_text_between(my_candidate)

' has developed a multivendor alliancewith Dell EMC, Intel, Juniper Networks, Red Hat and '

In [19]:
for t in get_between_tokens(my_candidate) :
    print(t)

has
developed
a
multivendor
alliancewith
dell
emc
,
intel
,
juniper
networks
,
red
hat
and


In [20]:
for t in get_left_tokens(my_candidate, window=5) :
    print(t)

In [21]:
for t in get_right_tokens(my_candidate, window=5) :
    print(t)

,
aiming
to
accelerate
network


In [22]:
get_tagged_text(my_candidate)

'{{A}} has developed a multivendor alliancewith Dell EMC, Intel, Juniper Networks, Red Hat and {{B}}, aiming to accelerate network functions virtualization (NFV) deployments in service provider networks\n'

In [23]:
get_text_splits(my_candidate)

['',
 '{{A}}',
 ' has developed a multivendor alliancewith Dell EMC, Intel, Juniper Networks, Red Hat and ',
 '{{B}}',
 ', aiming to accelerate network functions virtualization (NFV) deployments in service provider networks\n']